# 📚🪓⏱️ **`Matvejev word Shredder`** (MwS)
or the burial of `Byte-pair Encoding`

---

 * ### 🖱️ **One-click**
 * ### 🏷️ **Zero-label**
 * ### ⚡️ **Fast**
 * ### 🔪 **Surgical**
 * # 🧬🧠🧨 **`Morphemic Tokenizer`** ✂️🧩🧾
 ---



Этот проект посвящён улучшению классического `char-level Byte-pair encoding` (далее — `cBPE`) для аппроксимации морфемных разбиений. Наша задача состояла в существенной модификации и стабилизации паттернов, по которым производится слияние меньших токенов в токены большей длины на основе их совместной встречаемости.

Принципиальным в данной работе было сохрание unsupervised-природы оригинального `BPE` для удобства применения на больших неразмеченных текстовых данных.

**`Matvejev word Shredder`** при соответствующей настройке может существенно улучшить обощающую способность генеративных языковых моделей, гораздо эффективнее сокращая размер словаря, чем это делает большинство современных механизмов токенизации. Наибольшую ценность он будет представлять для морфологически богатых языков, а также языков с развитыми паттернами словообразования.

Перспективы, которые мы видим для **`MwS`**:

* Комбинация с **`Multi-Token Prediction`** (как, например, в *`DeepSeek R1`*). Возможность предсказывать сразу несколько (2-5) токенов, при учёте корректности морфемных границ позволит увеличить качество генерации при снижении времени генерации от **`Multi-Token Prediction`**.
* **Морфемные разбиения уменьшат размер эмбеддинга на токен**, что увеличит скорость обучения и инференса, либо **улучшит качество** при сохранении стандартных размеров. У стандартного `BPE` часты ситуации, когда одни токены представляют из себя **целые слова** или даже **бóльшие конструкции**, а другие - **наборы из байтов**, которые вообще могут быть не видны пользователю. Один токен в **`MwS`** будет представлять из себя последовательность из нескольких символов, что существенно снизит разброс кодируемой эмбеддингом информации. Каждая такая последовательность будет нести в себе меньше содержания, чем целое слово, из-за чего представление слова будет гораздо более чётким.
* В то же время, чёткость размеченных морфемных границ позволит языковым моделям меньшей размерности **быстрее усваивать грамматические и деривационные паттерны**, а также существенно **экономить на размере словаря**.

# Imports

In [1]:
%pip install --quiet datasets==2.15.0 pymorphy3==2.0.4 transformers==4.51.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 92.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 94.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2023.10.0 which is incompatible.


In [2]:
import json
import os
import math

from collections import Counter
from dataclasses import dataclass
from functools import lru_cache
from itertools import pairwise, chain
from math import log
from pathlib import Path
from typing import Dict, List, Optional, Set, Tuple

import pymorphy3
import regex as re
import huggingface_hub
from huggingface_hub import HfApi, PyTorchModelHubMixin, interpreter_login, snapshot_download
from huggingface_hub.utils import SoftTemporaryDirectory
from tempfile import TemporaryDirectory as SoftTemporaryDirectory
from tqdm.auto import tqdm, trange

In [3]:
from google.colab import userdata
huggingface_hub.login(token=userdata.get('hf_tok'))

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from string import punctuation

username = HfApi().whoami()["name"]
REPO_NAME = f"{username}/Matvejev_word_shredder"

print(f"Repository: '{REPO_NAME}'")
SEED = 0xC0FFEE

alphas ='абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
ALPHABET = set(fr' {alphas}{alphas.upper()}0123456789{punctuation}')
print(len(ALPHABET), f'{ALPHABET=}')

Repository: 'YourMasya/Matvejev_word_shredder'
109 ALPHABET={'э', '0', 'И', 'Ж', '_', 'б', ')', '&', 'Т', 'К', 'а', 'Г', '4', 'Х', '9', 'ф', '-', ',', 'ъ', ']', 'М', '{', 'В', '^', '/', '>', '=', '@', '%', ';', 'Е', '3', 'Р', ':', 'н', 'О', '"', '|', 'Ф', 'Ч', '?', '5', 'Д', 'в', '8', '~', '\\', 'й', 'Ш', 'Ь', 'м', 'ь', '<', ' ', '.', 'к', 'Н', 'З', 'Й', 'ю', 'я', "'", '*', 'Э', 'Ъ', '!', 'Щ', '`', 'Б', 'ш', 'Ы', 'о', 'Ц', '$', 'е', 'т', 'у', 'щ', 'х', '[', '}', '+', 'ж', 'ё', 'ц', '7', '6', 'А', 'Ё', 'Ю', '2', 'р', 'п', 'Л', '(', 'г', 'з', 'С', 'и', 'л', 'ы', 'У', 'Я', 'с', 'д', '1', 'ч', 'П', '#'}


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# Data and useful tools

Обучение токенизатора с помощью статистических инструментов на больших корпусах текстов для усмотрения морфологических и деривационных зависимостей видится нам абсолютно бесполезным. Данные в таком случае являются несбалансированными. Токенизатор сможет подогнаться под выборку, но что насчёт генеральной совокупности, включающей в себя все возможные формы данного слова, аналогичные формы других слов и однокоренные слова? При таком подходе успех маловероятен. Это противоречит конечной цели при тренировке любой языковой модели — увеличению обобщающей способности (желательно на множестве разнообразных задач).

Мы утверждаем, что именно поморфемная токенизация является ключевой предпосылкой для экономии и эффективности. Выделим основные типы проблем, возникающие при неправильной токенизации и распределим их по убыванию: от самой до наименее критичной.

1.   **Нарушены гарницы** морфем: ```['пер', 'ечит']```
2.   Несколько морфем объеденено в один **длинный** токен:  ```['перечит']``` = Переобучение.
3.   Составляющие одной морфемы **не объеденены** в один токен: ```['пе', ре', 'ч', 'ит']``` = Недообучение.

Некоторые формы слова могут отсутствовать в корпусе, несмотря на возможность появления. Впоследствии токенизатор будет разбивать однокоренные слова и даже формы одного слова так, что языковой модели будет тяжелее усмотреть важные граматические и лексические зависимости.

---




Сравним два типа разбиений и обсудим, почему одно лучше другого:
*   **Хорошие разбиения**:
```['пере', 'чит', 'ыва', 'ющ', 'ая', 'ся']```; ```['про', 'чит', 'а', 'вш', 'ая']```; ```['чит', 'а', 'ет']```

У слов выделен общий корень и окончание **-ая-**. У представлений 1 и 2 слов уже созданы предпосылки для усмотрения общих лексического значения, женского рода, единственного числа и именительного падежа. Корректны границы приставок, суффикса **-ыва-**, маркирующего несовершенный вид, постфикса **-ся**, маркирующего пассивный залог и свидетельствующих о различиях в информации. Форма действительного причастия прошедшего времени маркируется суффиксом **-вш-**, выделение суффикса **-ущ-** говорит о форме действительного причастия настоящего времени, образованного от глагола 1-го спряжения с основой на гласную. Последний фактор указывает на взаимную информацию с предыдущим суффиксом, что окажется полезным при генерации.

*   **Плохие разбиения**: ```['пе', 'речи', 'тыва', 'вшая', 'ся']```; ```['прочит', 'авши', 'й']```; ```['чита', 'вши', 'й']```

Данное разбиение содержит все 3 проблемы. Нарушение границ морфем, например, в токене **-тыва-** будет провоцировать модель локализовать в одном эмбеддинге как информацию из корня слова, так и из суффикса, что затруднит обобщение информации со словами, где суффикс **-ыва-** выделен корректно или выделенный токен содержит в себе часть этого суффикса либо помимо самого суффикса — часть другой морфемы. В токене -прочит- наблюдаем слияние приставки и суффикса, из-за чего модели понадобится существенно больше данных для нахождения сходств и различий с токенами **-почит-**, **-перечит-** **-недочит-** и т.д.


---



**Вывод**. Множество сходств и различий в морфемных составах слов при обучении модели будут связываться со сходствами и различиями в употреблениях — окружениях этих слов. Значение слова — и есть его употребление. И оно в том числе складывается из особенностей употребления морфем в контекстах, гораздо более широких, чем слово.

**Хороший датасет**, в соответствии с нашими потребностями должен будет удовлетворить 3 критериям:

1.   **Разнообразие форм** одного слова (прочитать, прочитали, прочитает, прочитайте и т.д.)
2.   **Разнообразие однокоренных** и **этимологически родственных** слов (перечитывать, отчитать, почитать, почитание, прочтение, читка и т.д.)
3.   Единичность вхождения / **уникальность** в датасете **каждого слова**

Критерии 1 и 2 будут удовлетворены при обучении на любом большом корпусе текстов (а на таких сейчас и учат LLM-ки). Мы же создадим игрушечный датасет из однокоренных слов и просклоняем их с помощью `pymorphy`

In [5]:
# Using GPT-4 pre-tokenization pattern
WHITESPACE_SPLITTER = re.compile(r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+""")

def decline_words(data: Set,
                  to_lower: bool=True):
  wordforms = set()
  morph = pymorphy3.MorphAnalyzer()

  for token in data:
    token = token.lower() if to_lower else token
    wordforms |= {p.word for p in morph.parse(token)[0].lexeme
    if p.word not in wordforms}
  return wordforms

In [18]:
toy_data = {
  'писать', 'написать', 'написывать', 'прописать', 'прописывать',
  'прописаться', 'прописываться', 'написаться', 'написываться', 'подписать',
  'подписывать', 'подписаться', 'подписываться', 'синий', 'синеть', 'осенний',
  'синь', 'синька', 'осень', 'пописать', 'пописывать', 'читать', 'прочитать',
  'начитаться', 'дочитать', 'прочитывать', 'прочитываться', 'прочитаться',
  'читаться', 'дочитывать', 'дочитаться', 'дочитываться', 'зачитать',
  'зачитаться', 'зачитывать', 'зачитываться', 'почитать', 'почитывать',
  'почитаться', 'писька', 'писюлька', 'перечитать', 'переписать',
  'переписывать', 'перечитывать', 'переписываться', 'переписаться',
  'перечитаться', 'перечитываться', 'читка', 'перепись', 'пропись'
}

toy_data = decline_words(toy_data)
toy_data = [" ".join(toy_data)]
print(WHITESPACE_SPLITTER.findall(toy_data[0]))

['прочитываемыми', ' подписавшейся', ' прочитывающимся', ' пописываемый', ' зачитывавшихся', ' дочитавшимся', ' прочитающуюся', ' зачитавшееся', ' подписываться', ' подписало', ' перепишите', ' прописываемых', ' дочитало', ' переписывавших', ' дочитанными', ' зачитывающая', ' почитает', ' зачитывавшегося', ' прописанного', ' зачитанного', ' пописаны', ' подписывали', ' зачитались', ' перечитывавшимися', ' зачитывайся', ' подписывающий', ' написывающемся', ' почитываемое', ' подписавшие', ' синевшее', ' перечитан', ' перепишешься', ' дочитавшем', ' напишущейся', ' подписанные', ' перепишущаяся', ' написалось', ' переписало', ' написавшихся', ' переписывающегося', ' зачитывавшем', ' перечитывающихся', ' написываемою', ' подписываете', ' прочитали', ' пописавши', ' перепишут', ' перечитавшегося', ' написавшим', ' напишущийся', ' читанное', ' написываемым', ' подписался', ' подписывающем', ' переписывавшим', ' прописывалось', ' прочитавшей', ' синевшею', ' перечитываемыми', ' попишет', ' п

# **`Matvejev Word Shredder`** architecture

Наш токенизатор является модификацией над `cBPE`. Алгоритм функционирует **без учителя**. Мы реализовываем **кастомный функционал качества**, аппроксимирующмй морфемные разбиения в **хорошем датасете** (см. критерии выше). Его предстоит жадно максимизировать. Он включает:
  *   **Pointwise Mutual Information (PMI)**. Одна из самых популярных в компьютерной лингвистике мер ассоциации, используется для выделения коллокаций. Измеряет взаимозависимость между двумя токенами, показывая, сколько информации один токен предоставляет о другом.
  *   **T-score**. Также очень популярная мера ассоциации для выделения коллокаций. Используется нами для копенсации проблемы PMI к присвоению редким биграммам больших значений. T-score как бы говорит относительно таких биграмм: "Возможно стоит слить два этих токена в один, но у нас недостаточно доказательств, чтобы быть уверенными в том, что токены, получившие большое значение PMI действительно являются хорошими кандидатами на слияние!"
  *   **Information Gain**. Используется как функционал качества при обучении классификатора, основанного на решающих деревьях. В нашем случае измеряет снижение неопределенности при слиянии двух совместно встречающихся токенов в один. Именна эта мера "чувствует" границы морфем.

Чем больше значение этих мер, тем сильнее мы хотим слить 2 токена в 1.


---


Также рассчитываются:

  *   **Нормализация на длину** токена-кандидата на слияние, в общих чертах повторяющая ту, что используется в алгоритме **BM-25**. Здесь вводится штраф за слишком длиные токены.
  *   **Геометрическое среднее**. Считается дважды: 1) между **PMI** и **T-score**. Геометрическое среднее, в отличии от среднего арифметического, позволяет избавиться от случаев, когда большое значение одного члена и маленькое значение другого члена превращается в довольно большое значение после усреднения. Значения после геометрического усреднения не могут быть высокими, если один из членов имеет маленькое значение. Т.е. такая операция как бы показывает, что мы хотим одновременно растить и **PMI**, и **T-score** в первом случае, а также значения **нормализованной аггрегированной меры** и **Information Gain** во втором.

---



Помимо этого, мы используем некоторые **эвристики**, стабилизирующие обучение алгоритма:


*   Пропускаем кандидатов на слияние, длина которых больше, чем установленный порог (дефолтно, 5)
*   Пропускаем кандидатов на слияние, абсолютное значение длины которых отличается от корня из средней длины токена больше, чем на установленный порог (дефолтно, 2)
*   **Штраф за длину** вычисляется как **корень из разности квадратов длин**: Из квадрата длины токена-кандидата на слияние вычитаем квадраты длин составляющих его токенов. И берём из получившегося числа квадратный корень. Так, токен-кандидат на слияние, допустим `abcd`, будет получать менее жёсткий штраф, если будет состоять из токенов длины 3 и 1 (`a` и `bcd` или `abc` и `d`), чем если бы он состоял из токенов длины 2 и 2 (`ab` и `cd`). **Объяснение:** На практике было замечено, что границы морфем чаще нарушались при слияниях второго типа, чем первого.
*   Обучение останавливается, если значение функционала качества топ-пары равно 0.





In [152]:
def get_pairs(word: Tuple[str]
) -> Set[Tuple[str, str]]:
    """Getting unique pairs of word unigrams"""
    return set(pairwise(word))

def calc_merge_coef(
    bi_counts: Dict[Tuple[str, str], int],
    uni_counts: Dict[str, int],
    len_norm_factor: float = 2.0,
    log_base: int = 2,
    gap_threshold: float = 1.0,
    max_merge_len: int = 5,
    max_len_delta: int = 2,
    report: bool = True
) -> Dict[Tuple[str, str], float]:
    """Calculating the merge coefficient used in tokenizer"""
    merge_dict = {}
    square, root = 2, 0.5
    norm_agg_assoc = []
    if log_base <= 1:
      raise ValueError('Logarithm base should be above 1')

    # Compute the arithmetic mean of unigram lengths for
    # furhter use in length penalization formula
    n_unigrams, n_bigrams = sum(uni_counts.values()), sum(bi_counts.values())
    len_count_sum = sum(len(token) * count for token, count in uni_counts.items())
    avg_uni_len = len_count_sum / n_unigrams

    # Define a very small value to avoid ZeroDivisionError in some computations
    eps = 1e-24

    for bigram, bi_count in bi_counts.items():
      left, right = bigram

      # We won't penalize for space length in the left token
      L_len = max(len(left.lstrip()), 1)
      R_len = len(right)
      pair_len = L_len + R_len

      # Skip bigram and do not count merge coefficient if: 1) candidate length
      # is above max_merge_len or 2) the absolute difference between
      # candidate length and average unigram length is above max_len_delta
      if (pair_len > max_merge_len or
          abs(pair_len - (avg_uni_len ** root)) > max_len_delta):
        continue

      # Compute length penalty for further use
      sq_len_penalty = pair_len ** square - L_len ** square - R_len ** square

      # Calculate probabilities for further computations
      # Raise to a power 0.75 to avoid high PMI of rare unigram problem
      left_prob = uni_counts[left] / n_unigrams
      right_prob = uni_counts[right] ** 0.75 / n_unigrams ** 0.75
      bi_prob = bi_count / n_bigrams

      # Calculate T-score
      t_score = (
          (bi_count - (uni_counts[left] * right_prob)) /
          (bi_count ** root)
      )
      # Compute a part of PMI without taking a logarithm
      pre_pmi = bi_prob / (left_prob * right_prob)

      # Geometric mean of PMI and t-score.
      # Adding 1 to t-score for taking a logarithm
      assoc_agg = root * (
          log(max(eps, pre_pmi), log_base) +
          log(max(1 + eps, t_score + 1), log_base)
      )

      # Compute entropy of the next (merge candidate)
      # and previous (unigrams) state.
      H_bi = -bi_prob * log(bi_prob, log_base)
      H_unigrams = -right_prob * log(right_prob, log_base) - left_prob * log(left_prob, log_base)

      # Compute information gain
      info_gain = H_unigrams - H_bi

      # Use length penalization similar to BM-25 one
      relative_len_pen = (sq_len_penalty ** root) / avg_uni_len
      penalized_score = assoc_agg - log(log_base - len_norm_factor +
                                    len_norm_factor * relative_len_pen,
                                    log_base)

      # The sum of logarithms = the logarithm of multiplication
      merge_coef = max(penalized_score + info_gain, 0)

      # Add a merge coefficient of the bigram to the dictionary
      merge_dict[bigram] = merge_coef
      if report:
        print(bigram)
        print(f'{avg_uni_len=}; {sq_len_penalty=}; {relative_len_pen=:.3f}')
        print(f'{pre_pmi=:.3f}; {t_score=:.3f}')
        print(f'{assoc_agg=:.3f} before length penalty')
        print(f'{penalized_score=:.3f}')
        print(f'{H_bi=:.3f}; {H_unigrams=:.3f}; {info_gain=:.3f}')
        print(f'{merge_coef=:.3f}')
        print('=' * 50)
    if merge_dict and report:
      print(f'''MIN MAX VALS:
      {min(merge_dict.values()):.3f};
      {max(merge_dict.values()):.3f}''')
      print('=' * 50)
    return merge_dict

def recalculate_counts(words_by_tokens: Dict[Tuple[str, ...], int]
  ) -> Tuple[Dict[str, int], Dict[Tuple[str, str], int]]:
    """Recalculating unigram and bigram counts"""
    unigram_counts = Counter()
    bigram_counts = Counter()

    for word, count in words_by_tokens.items():
      word_len = len(word)
      for unigram in word:
        unigram_counts[unigram] += count

      for i in range(word_len - 1):
        bigram = (word[i], word[i + 1])
        bigram_counts[bigram] += count

    return unigram_counts, bigram_counts

In [153]:
def merge(
    merge_pair: Tuple[str, str],
    words_by_tokens: Dict[Tuple[str, ...], int],
    len_norm_factor: float = 2.0,
    log_base: int = 2,
    gap_threshold: float = 1.0,
    max_merge_len: int = 5,
    max_len_delta: int = 2
):
    """Merges a given pair of tokens and updates corresponding stats"""
    new_token = merge_pair[0] + merge_pair[1]
    updated_words = {}

    # Process each word containing the bigram
    for word, count in list(words_by_tokens.items()):
        # Skip words that don't contain the merge pair
        found = False
        for i in range(len(word) - 1):
            if (word[i], word[i + 1]) == merge_pair:
                found = True
                break
        if not found:
            continue

        # Remove original word
        del words_by_tokens[word]

        # Build new word with merged token
        new_word = []
        i = 0
        while i < len(word):
            if i < len(word) - 1 and (word[i], word[i + 1]) == merge_pair:
                new_word.append(new_token)
                i += 2
            else:
                new_word.append(word[i])
                i += 1

        # Add new version of word
        new_word_tup = tuple(new_word)
        words_by_tokens[new_word_tup] = words_by_tokens.get(new_word_tup, 0) + count

    # Recalculate all counts
    unigram_counts, bigram_counts = recalculate_counts(words_by_tokens)
    merge_info_dict = calc_merge_coef(bigram_counts, unigram_counts,
                                      len_norm_factor, log_base, gap_threshold,
                                      max_merge_len, max_len_delta)
    return merge_info_dict, words_by_tokens, unigram_counts, bigram_counts

In [154]:
def train(
    data: List[str],
    n_merges: int,
    special_tokens: List[str] = None,
    alphabet: Set[str] = None,
    len_norm_factor: float = 2.0,
    log_base: int = 2,
    gap_threshold: float = 1.0,
    max_merge_len: int = 5,
    max_len_delta: int = 2,
    stop_threshold: float = 2
):
    """Trains BPE tokenizer on the data passed"""
    if n_merges <= 0:
        raise ValueError("The number of merges should be greater than zero")
    special_tokens = special_tokens or []

    # Initialize vocabulary
    chars = sorted(list(alphabet))
    id2token = {idx: char for idx, char in enumerate(chars)}
    merges = []

    # Tokenize data and count words
    words_by_tokens = {}
    for sample in data:
        words = WHITESPACE_SPLITTER.findall(sample)
        for word in words:
            tokens = tuple(word)
            words_by_tokens[tokens] = words_by_tokens.get(tokens, 0) + 1

    # Calculate initial counts
    unigram_counts, bigram_counts = recalculate_counts(words_by_tokens)

    # Initial probabilities
    merge_info_dict = calc_merge_coef(bigram_counts, unigram_counts,
                                      len_norm_factor, log_base, gap_threshold,
                                      max_merge_len, max_len_delta)

    # Build vocabulary
    while len(id2token) < n_merges + len(set(alphabet)) + len(special_tokens):
        if not merge_info_dict:
            break

        # Select pair with highest merge coefficient
        top_pair = max(merge_info_dict, key=merge_info_dict.get)
        new_token = top_pair[0] + top_pair[1]
        top_merge_coef = merge_info_dict[top_pair]

        # Stop execution if top_pair merge coefficient
        # below the threshold or equals 0
        if stop_threshold and isinstance(stop_threshold, float) and \
        top_merge_coef <= stop_threshold or top_merge_coef == 0:
            break

        # Skip if token already exists
        if new_token in id2token.values():
            del merge_info_dict[top_pair]
            continue

        # Record merge
        id2token[len(id2token)] = new_token
        merges.append(top_pair)

        # Perform merge and get updated state
        merge_info_dict, words_by_tokens, unigram_counts, bigram_counts = merge(
            top_pair, words_by_tokens, len_norm_factor, log_base, gap_threshold,
            max_merge_len, max_len_delta
        )

    # Add special tokens
    for token in special_tokens:
        id2token[len(id2token)] = token

    return {token: idx for idx, token in id2token.items()}, merges, merge_info_dict

In [155]:
class MatvejevWordShredder:
    def __init__(self, vocab: Dict[str, int],
                 merges: List[Tuple[str, str]],
                 eos_token: str = "[EOS]"):
        if eos_token not in vocab:
            raise ValueError("EOS token missing from vocabulary")
        self.token2id = vocab
        self.id2token = {v: k for k, v in self.token2id.items()}
        self.eos_token = eos_token
        self.eos_token_id = vocab[eos_token]
        self.merges = merges
        self.bpe_ranks = {pair: i for i, pair in enumerate(merges)}
        self._vocab_tokens = sorted(vocab.keys(), key=len, reverse=True)

    @lru_cache(maxsize=10000)
    def bpe(self, token: str) -> Tuple[str]:
        if len(token) == 1:
            return (token,)

        word = list(token)
        while len(word) > 1:
            pairs = list(zip(word[:-1], word[1:]))
            bigram = min(
                pairs,
                key=lambda pair: self.bpe_ranks.get(pair, float('inf'))
            )
            if bigram not in self.bpe_ranks:
                break

            new_word = []
            i = 0
            while i < len(word):
                if i < len(word) - 1 and (word[i], word[i+1]) == bigram:
                    new_word.append(word[i] + word[i+1])
                    i += 2
                else:
                    new_word.append(word[i])
                    i += 1
            word = new_word
        return tuple(word)

    def encode(self, text: str, add_eos_token: bool = False) -> List[int]:
        token_ids = []
        words = WHITESPACE_SPLITTER.findall(text)
        for word in words:
            token_ids.extend(
                self.token2id[token]
                for token in self.bpe(word)
            )

        if add_eos_token:
            token_ids.append(self.eos_token_id)

        return token_ids

    def decode(self, token_ids: List[int]) -> str:
        return [self.id2token.get(t_id, "[UNK]") for t_id in token_ids]

    def push_to_hub(self, repo_id, *, private=None, token=None):
        api = HfApi()
        repo_id = api.create_repo(
            repo_id=repo_id,
            token=token,
            private=private,
            exist_ok=True
        ).repo_id

        with SoftTemporaryDirectory() as tmp:
            save_dir = Path(tmp) / repo_id
            save_dir.mkdir(parents=True)

            # Save vocabulary and merges
            (save_dir / "vocabulary.json").write_text(
                json.dumps(self.token2id, indent=2)
            )
            (save_dir / "merges.json").write_text(
                json.dumps({"merges": self.merges})
            )

            api.upload_folder(
                repo_id=repo_id,
                folder_path=save_dir,
                token=token
            )

    @classmethod
    def from_pretrained(cls, pretrained_model_name_or_path, *, token=None, **kwargs):
        if not os.path.isdir(pretrained_model_name_or_path):
            model_path = snapshot_download(
                repo_id=pretrained_model_name_or_path,
                token=token
            )
        else:
            model_path = pretrained_model_name_or_path

        model_path = Path(model_path)
        vocab = json.loads((model_path / "vocabulary.json").read_text())
        merges_data = json.loads((model_path / "merges.json").read_text())

        return cls(
            vocab,
            [tuple(pair) for pair in merges_data["merges"]],
            **kwargs
        )

# Results & Metrics

In [156]:
# The number of alphabet symbols
len(set(ALPHABET))

109

In [163]:
# Explicitly place default values as input
vocab, merges, merge_dict = train(toy_data,
                                  n_merges=116,
                                  special_tokens=['[EOS]'],
                                  alphabet=ALPHABET,
                                  len_norm_factor=2.0,
                                  log_base=2,
                                  gap_threshold=1.0,
                                  max_merge_len=5,
                                  max_len_delta=2,
                                  stop_threshold=2)

Выходные данные были обрезаны до нескольких последних строк (5000).
penalized_score=0.879
H_bi=0.002; H_unigrams=0.112; info_gain=0.111
merge_coef=0.989
('нн', 'и')
avg_uni_len=2.5419204222492895; sq_len_penalty=4; relative_len_pen=0.787
pre_pmi=5.426; t_score=0.767
assoc_agg=1.631 before length penalty
penalized_score=0.977
H_bi=0.002; H_unigrams=0.104; info_gain=0.102
merge_coef=1.079
('а', 'ла')
avg_uni_len=2.5419204222492895; sq_len_penalty=4; relative_len_pen=0.787
pre_pmi=4.311; t_score=3.390
assoc_agg=2.121 before length penalty
penalized_score=1.467
H_bi=0.025; H_unigrams=0.287; info_gain=0.263
merge_coef=1.730
('е', 'ла')
avg_uni_len=2.5419204222492895; sq_len_penalty=4; relative_len_pen=0.787
pre_pmi=1.847; t_score=0.316
assoc_agg=0.641 before length penalty
penalized_score=-0.014
H_bi=0.002; H_unigrams=0.131; info_gain=0.130
merge_coef=0.116
('ан', 'а')
avg_uni_len=2.5419204222492895; sq_len_penalty=4; relative_len_pen=0.787
pre_pmi=6.931; t_score=2.712
assoc_agg=2.343 befor

In [164]:
tokenizer = MatvejevWordShredder(vocab, merges)
ids = [tokenizer.encode(word) for word in toy_data]
reversed_text = [tokenizer.decode(inx) for inx in ids]
print(*sorted([word for word in reversed_text], reverse=True), sep='\n')

['про', 'чит', 'ыва', 'ем', 'ыми', ' под', 'пис', 'авш', 'ей', 'ся', ' про', 'чит', 'ыва', 'ющ', 'им', 'ся', ' по', 'пис', 'ыва', 'ем', 'ый', ' за', 'чит', 'ыва', 'вш', 'их', 'ся', ' до', 'чит', 'авш', 'им', 'ся', ' про', 'чит', 'ающ', 'ую', 'ся', ' за', 'чит', 'авш', 'ее', 'ся', ' под', 'пис', 'ыва', 'ть', 'ся', ' под', 'пис', 'а', 'ло', ' пе', 'ре', 'пиш', 'ите', ' про', 'пис', 'ыва', 'ем', 'ых', ' до', 'чит', 'а', 'ло', ' пе', 'ре', 'пис', 'ыва', 'вш', 'их', ' до', 'чит', 'анн', 'ыми', ' за', 'чит', 'ыва', 'ющ', 'ая', ' по', 'чит', 'а', 'ет', ' за', 'чит', 'ыва', 'вш', 'его', 'ся', ' про', 'пис', 'анн', 'ого', ' за', 'чит', 'анн', 'ого', ' по', 'пис', 'а', 'ны', ' под', 'пис', 'ыва', 'ли', ' за', 'чит', 'а', 'ли', 'сь', ' пе', 'ре', 'чит', 'ыва', 'вш', 'ими', 'ся', ' за', 'чит', 'ыва', 'й', 'ся', ' под', 'пис', 'ыва', 'ющ', 'ий', ' на', 'пис', 'ыва', 'ющ', 'ем', 'ся', ' по', 'чит', 'ыва', 'ем', 'ое', ' под', 'пис', 'авш', 'ие', ' ', 'син', 'е', 'вш', 'ее', ' пе', 'ре', 'чит', 'ан', 

Есть небольшие огрехи, но в целом - очень неплохо !!!

In [165]:
print(tokenizer.decode(tokenizer.encode(' переписывалась')))
print(tokenizer.decode(tokenizer.encode(' подписавшийся')))
print(tokenizer.decode(tokenizer.encode(' прочитывать')))
print(tokenizer.decode(tokenizer.encode(' почитаться')))

[' пе', 'ре', 'пис', 'ыва', 'ла', 'сь']
[' под', 'пис', 'авш', 'ий', 'ся']
[' про', 'чит', 'ыва', 'ть']
[' по', 'чит', 'а', 'ть', 'ся']


In [166]:
print(len(merges), merges)

# this line is used to manually adjust the optimal number of merges
# print(merges.index(('син', 'ь')))

117 [('ы', 'в'), ('в', 'ш'), ('ю', 'щ'), ('ч', 'и'), ('чи', 'т'), (' ', 'п'), ('п', 'и'), ('с', 'я'), ('пи', 'с'), ('с', 'ь'), ('пи', 'ш'), ('ш', 'ь'), ('ь', 'к'), ('т', 'ь'), (' ', 'з'), ('у', 'щ'), ('у', 'ю'), ('г', 'о'), ('ыв', 'а'), (' з', 'а'), ('р', 'о'), ('р', 'е'), (' п', 'о'), ('д', 'о'), (' по', 'д'), (' п', 'ро'), ('п', 'ро'), (' п', 'е'), (' п', 'и'), ('е', 'м'), (' пи', 'с'), (' пи', 'ш'), ('с', 'и'), ('и', 'м'), ('ы', 'м'), ('о', 'м'), ('о', 'с'), ('и', 'х'), ('ы', 'х'), ('си', 'н'), (' ', 'н'), (' н', 'а'), ('н', 'н'), (' ', 'до'), ('т', 'е'), ('ю', 'т'), ('ом', 'у'), ('л', 'о'), ('н', 'ы'), ('ь', 'ю'), ('ы', 'й'), ('ы', 'е'), ('й', 'ш'), ('им', 'и'), ('и', 'й'), ('л', 'и'), ('м', 'и'), ('и', 'е'), ('ым', 'и'), ('о', 'ю'), ('о', 'й'), ('н', 'о'), ('о', 'го'), ('о', 'е'), ('е', 'ю'), ('е', 'й'), ('й', 'те'), ('е', 'т'), ('у', 'т'), ('ос', 'е'), ('е', 'го'), ('е', 'е'), ('е', 'шь'), ('е', 'те'), ('е', 'йш'), ('ем', 'у'), ('ю', 'л'), ('а', 'вш'), ('а', 'я'), ('а', 'нн'), ('

In [167]:
tokenizer = MatvejevWordShredder(vocab, merges)
tokenizer.push_to_hub(REPO_NAME)

In [168]:
tokenizer = MatvejevWordShredder.from_pretrained(REPO_NAME)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

merges.json: 0.00B [00:00, ?B/s]

vocabulary.json: 0.00B [00:00, ?B/s]

# ***To be continued...***
В скором времени мы планируем создать полноценную библиотеку, производящую токенизацию, измерить качество разбиений с помощью метрик (а не на глаз), выложить в открытый доступ предобученные токенизаторы для нескольких крупных морфологически богатых языков, и сравнить модели, обученные на наших токенах с моделями, обученными на стандартном `cBPE`, заодно применив в них `Multi-Token Prediction`.